In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
def login(login, psswd, driver):
    driver.get("https://reservation.42network.org/signin")
    login_field = driver.find_element_by_id("user_login")
    psswd_field = driver.find_element_by_id("user_password")
    login_field.clear()
    login_field.send_keys(login)
    psswd_field.clear()
    psswd_field.send_keys(psswd)
    psswd_field.send_keys(Keys.RETURN)
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//td[@class='fc-widget-content']")))

In [3]:
def logout(driver):
    driver.find_elemt_by_xpath("//*[@id='navbar-main']/div[2]/a[2]").click()
    driver.find_element_by_xpath("//span[text()='logout']").click()    

In [4]:
def get_week_days(driver):
    return driver.find_elements_by_xpath("//div[@class='fc-content-skeleton']//div[@class='fc-event-container']")

In [5]:
def get_elements_in_day(day_element):
    return day_element.find_elements_by_xpath(".//a")

In [6]:
slot_times = ['08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00']

In [41]:
class Button():
    def __init__(self, element = None):
        self.element = element
        self.floor = self.get_floor()
        self.hour = self.get_hour()
        self.reserved = self.get_reserved()


    def get_floor(self):
        try:
            self.floor = self.element.find_element_by_class_name("fc-title").text
        except:
            self.floor = None
            print("SELF FLOOR NONE")
        
        return self.floor


    def get_reserved(self):
        try:
            self.reserved = self.element.find_element_by_xpath(".//i[@class='mdi mdi-calendar-check fc-event-is-subscribe-icon ']") is not None
        except:
            self.reserved = False
        
        return self.reserved


    def get_hour(self):
        try:
            self.hour = self.element.find_element_by_xpath(".//div[@class='fc-time']").get_attribute("data-start")
        except:
            self.hour = None
            print("SELF HOUR = NONE")
    
        return (self.hour)
        

    def __str__(self):
        return (f"{self.hour} {self.floor}, reserved: {self.reserved}")



class Slot():
    def __init__(self, day_element, hour: str, desired = True, date = None):
        self.hour = hour
        self.day_element = day_element
        self.buttons = {}
        self.reserved = False
        self.reserved_floor = None
        self.desired = desired
        self.date = date
        self.set_buttons()
        self.set_reserved()


    def __str__(self):
        return (f"{self.hour}, reserved: {self.reserved_floor}, buttons: {[str(x) for x in self.buttons.values()]}")


    def get_buttons(self):
        try:
            return [Button(x) for x in self.day_element.find_elements_by_xpath(f".//a/div/div[@data-start='{self.hour}']/../..")]
        except Exception as e:
            print(f"no elements found at {self.hour}")
            print(e)
            return []
        

    def set_buttons(self):
        for but in self.get_buttons():
            self.buttons[but.floor] = but


# REFRESH RESERVED VALEUS ??
    def set_reserved(self):
        reserved = False
        for but in self.buttons.values():
            if (but.reserved):
                self.reserved = True
                self.reserved_floor = but.floor

    

class Day():
    def __init__(self, day_element, wanted_times = slot_times):
        self.slots = {}
        self.get_slots(day_element, wanted_times)


    def get_slots(self, day_element, wanted_times):
        for hour in wanted_times:
            self.slots[hour] = Slot(day_element, hour)

In [19]:
# driver = webdriver.Chrome("./chromedriver_linux")
driver = webdriver.Firefox()
login("jbarment", "69@TheEelHouse!", driver)
weekdays = get_week_days(driver)
mday = weekdays[4]

In [43]:
friday = Day(mday, slot_times)

In [47]:
for slut in friday.slots.values():
    print(slut)

08:00, reserved: None, buttons: ['08:00 E1, reserved: False', '08:00 E2, reserved: False', '08:00 TDM, reserved: False']
09:00, reserved: None, buttons: ['09:00 E1, reserved: False', '09:00 E2, reserved: False', '09:00 TDM, reserved: False']
10:00, reserved: E2, buttons: ['10:00 EXAM E1, reserved: False', '10:00 E2, reserved: True', '10:00 TDM, reserved: False']
11:00, reserved: E2, buttons: ['11:00 E2, reserved: True', '11:00 TDM, reserved: False']
12:00, reserved: E2, buttons: ['12:00 E1, reserved: False', '12:00 E2, reserved: True', '12:00 TDM, reserved: False']
13:00, reserved: E2, buttons: ['13:00 E1, reserved: False', '13:00 E2, reserved: True', '13:00 TDM, reserved: False']
14:00, reserved: E2, buttons: ['14:00 E1, reserved: False', '14:00 E2, reserved: True', '14:00 TDM, reserved: False']
15:00, reserved: E2, buttons: ['15:00 E1, reserved: False', '15:00 E2, reserved: True', '15:00 TDM, reserved: False']
16:00, reserved: E2, buttons: ['16:00 E1, reserved: False', '16:00 E2, res

In [ ]:
class wanted_slots():
    def __init__(self, monday_start = None, monday_end = None, tuesday_start = None, tuesday_end = None, wednesday_start = None, wednesday_end = None, thursday_start = None,
                 thursday_end = None, friday_start = None, friday_end = None, saturday_start = None, saturday_end = None, sunday_start = None, sunday_end = None):
        self.monday_start = monday_start
        self.monday_end = monday_end
        self.tuesday_start = tuesday_start
        self.tuesday_end = tuesday_end
        self.wednesday_start = wednesday_start
        self.wednesday_end = wednesday_end
        self.thursday_start = thursday_start
        self.thursday_end = thursday_end
        self.friday_start = friday_start
        self.friday_end = friday_end
        self.saturday_start = saturday_start
        self.saturday_end = saturday_end
        self.sunday_start = sunday_start
        self.sunday_end = sunday_end
        self.array = [[monday_start, monday_end], [tuesday_start, tuesday_end], [wednesday_start, wednesday_end], [thursday_start, thursday_end], [friday_start, friday_end], [saturday_start, saturday_end], [sunday_start, sunday_end]]
        
class agenda():
    def __init__(self, wanted_slots):
        self.days = [[],[],[],[],[],[],[]]
        for day in range(7):
            print(day)